In [1]:
from time import time
import sys
sys.path.append('/models/StateFarmChallenge')
import os
import logging

from six.moves import xrange
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from submission import Submission

import math
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

First we need to process the data, by scaling an cropping each image.

In [60]:
DATA_FILE_PATH = '/data/SFC/driver_imgs_list.csv'

NUM_CLASSES = 10
batch_size = 128
patch_size = 10
depth1 = 32
depth2 = 64
depth3 = 64
num_hidden = 512
learning_rate = .01
num_channels = 1
image_size = 256 * 256
num_epochs = 2
beta = .01


def inference(images, keep_prob):
    """
    Args:
    images: Images placeholder, from inputs().
    Returns:
    softmax_linear: Output tensor with the computed logits.
    """

#     Conv1 /w pooling
    with tf.name_scope('conv1') as scope:
        weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, num_channels, depth1], stddev=0.01))
        biases = tf.Variable(tf.zeros([depth1]))
        conv = tf.nn.conv2d(images, weights, [1, 1, 1 ,1], padding='SAME', name='conv1')
        conv1 = tf.nn.relu(conv, name=scope)
        
    pool1 = tf.nn.avg_pool(conv1, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME', name='pool1')
    norm1 = tf.nn.local_response_normalization(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

#     Conv2 /w pooling 
    with tf.name_scope('conv2') as scope:
        weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, depth1, depth2], stddev=0.01))
        biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
        conv = tf.nn.conv2d(norm1, weights, [1, 1, 1, 1], padding='SAME', name='conv2')
        conv2 = tf.nn.relu(conv, name=scope)
    
    pool2 = tf.nn.avg_pool(conv2, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME', name='pool2')
    norm2 = tf.nn.local_response_normalization(pool2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm2')
    
#     Conv3 /w pooling
    with tf.name_scope('conv3') as scope:
        weights = tf.Variable(tf.truncated_normal(
              [patch_size, patch_size, depth2, depth3], stddev=0.01))
        biases = tf.Variable(tf.constant(1.0, shape=[depth3]))
        conv = tf.nn.conv2d(norm2, weights, [1, 1, 1, 1], padding='SAME', name='conv3')
        conv3 = tf.nn.relu(conv, name=scope)
    
    pool3 = tf.nn.avg_pool(conv3, [1, 6, 6, 1], [1, 2, 2, 1], padding='SAME', name='pool3')
    norm3 = tf.nn.local_response_normalization(pool3, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm3') 
    shape = norm3.get_shape().as_list()
    reshape = tf.reshape(norm3, [shape[0], shape[1] * shape[2] * shape[3]])
        
        
#     local4
    with tf.name_scope('local3') as scope:
        weights = tf.Variable(
            tf.truncated_normal([image_size, num_hidden],
                                stddev=1.0 / math.sqrt(float(num_hidden))))

        biases = tf.Variable(tf.zeros([num_hidden]))
        z = tf.matmul(reshape, weights) + biases
        local4 = tf.nn.relu(z, name=scope)

#     local5
    with tf.name_scope('local4') as scope:
        weights = tf.Variable(
            tf.truncated_normal([num_hidden, num_hidden],
                                stddev=1.0 / math.sqrt(float(num_hidden))), name='weights_5')

        biases = tf.Variable(tf.zeros([num_hidden]),name='biases_5')
        z = tf.matmul(local4, weights) + biases
        local5 = tf.nn.relu(z, name=scope)
        drop = tf.nn.dropout(local5, keep_prob)
        
        
    # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([num_hidden, NUM_CLASSES],
                                stddev=1.0 / math.sqrt(float(num_hidden))), name='weights_s')

        biases = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases_s')
        logits = tf.matmul(drop, weights) + biases
        
    return logits


def loss(logits, labels):
    """"
    Calculates the loss from the logits and the labels.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int64 - [batch_size].
    Returns:
    loss: Loss tensor of type float.
    """
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    loss_prime = loss + beta * tf.nn.l2_loss(logits)
    return loss_prime


def training(loss, learning_rate):
    """
    Sets up the training Ops.
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.
    Args:
    loss: Loss tensor, from loss().
    learning_rate: The learning rate to use for gradient descent.
    Returns:
    train_op: The Op for training.
    """
    # Add a scalar summary for the snapshot loss. (Tensor board)
    tf.scalar_summary('loss', loss)

    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # decay on our learning rate as we progress with training
    learn_rate = tf.train.exponential_decay(learning_rate, global_step, 1000, 0.8)
    tf.scalar_summary('learn_rate', learning_rate)
    
    # Create the gradient Adam optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)

    return train_op


def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label.
    Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
    Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
    """
    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1)
    # of all logits for that example.
    correct = tf.nn.in_top_k(logits, labels, 1)
    # Return the number of true entries.
    return tf.reduce_sum(tf.cast(correct, tf.int32))



def decode_and_process_img(queue):
    # Reader to read csv rows
    reader = tf.TextLineReader()
    key, value = reader.read(queue)
    
    record_defaults = [[''], [''], ['']]
    subject, label, img_file = tf.decode_csv(value, record_defaults=record_defaults)
    
    path = tf.cast('/data/SFC/imgs/train/' + label + '/' + img_file, tf.string)
    img_file = tf.read_file(path)
    img_data = tf.image.decode_jpeg(img_file, channels=1)
    resized = tf.image.resize_images(img_data, 256, 256)
#    scaled_croped = tf.image.crop_to_bounding_box(scaled, 0, 43, 256, 256)

    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = tf.cast(resized, tf.float32) * (1. / 255) - 0.5

    return image, label

    
def inputs(batch_size, num_epochs):
    with tf.name_scope('input'):
        # Create queue for csv file
        file_queue = tf.train.string_input_producer([DATA_FILE_PATH], num_epochs=num_epochs)

        image, label = decode_and_process_img(file_queue)
        
        images, labels = tf.train.shuffle_batch([image, label], batch_size=batch_size, 
                                                num_threads=2,
                                                capacity=1000 + 3 * batch_size,
                                                min_after_dequeue=1000)
    
    return images, labels

    


def run_training():
    tf.reset_default_graph()
    with tf.Graph().as_default():
        
        keep_prob = tf.placeholder(tf.float32)
        
        images, labels = inputs(batch_size, num_epochs)
        
        logits = inference(images, keep_prob)
                
        loss_op = loss(logits, labels)
        
        train_op = training(loss_op, learning_rate)

        init_op = tf.initialize_all_variables()


        sess = tf.Session()
        sess.as_default()
        
        sess.run(init_op)

        # Start populating the filename queue.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        print(path.eval(session=sess))
        print(label.eval(session=sess))
        print(image.eval(session=sess))
        
#         for i in range(1):
#             # Retrieve a single instance:
#             _, ls = sess.run([train_op, loss_op], feed_dict={keep_prob : 0.5})
#             print(ls)
#             print(len(labels))


        coord.request_stop()
        coord.join(threads)
        sess.close()
        
run_training()

/data/SFC/imgs/train/c0/img_44733.jpg
c0
[[[-0.43725491]
  [-0.43725491]
  [-0.43725491]
  ..., 
  [ 0.5       ]
  [ 0.49607849]
  [ 0.49215692]]

 [[-0.43725491]
  [-0.43725491]
  [-0.43725491]
  ..., 
  [ 0.5       ]
  [ 0.49607849]
  [ 0.49387258]]

 [[-0.43725491]
  [-0.43725491]
  [-0.43725491]
  ..., 
  [ 0.49607849]
  [ 0.49607849]
  [ 0.49607849]]

 ..., 
 [[-0.31421566]
  [-0.36421567]
  [-0.41470587]
  ..., 
  [-0.4615196 ]
  [-0.45539215]
  [-0.4642157 ]]

 [[-0.44607842]
  [-0.22401959]
  [-0.09313723]
  ..., 
  [-0.45980391]
  [-0.44705883]
  [-0.45588234]]

 [[-0.49313724]
  [-0.39289215]
  [-0.17941174]
  ..., 
  [-0.45514706]
  [-0.43186274]
  [-0.38774508]]]


InvalidArgumentError: Field 1 in record 0 is not a valid int32: c0
	 [[Node: input/DecodeCSV = DecodeCSV[OUT_TYPE=[DT_STRING, DT_INT32, DT_STRING], field_delim=",", _device="/job:localhost/replica:0/task:0/cpu:0"](input/ReaderRead:1, input/DecodeCSV/record_defaults_0, input/DecodeCSV/record_defaults_1, input/DecodeCSV/record_defaults_2)]]
Caused by op u'input/DecodeCSV', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-